# Predict Radiative heating corresponding to given parameters

## Import package

In [20]:
import h5py
import numpy as np
from matplotlib import pyplot as plt

from metpy.calc import pressure_to_height_std
from metpy.units import units
from scipy.interpolate import interp1d

## Load data

In [21]:
# Load LRF
## Load LRF for T, q
with h5py.File("/work/b11209013/2025_Research/MSI/Rad_Stuff/LRF.h5", "r") as f:
    q_lw_lrf = np.array(f.get("q_lw"))
    q_sw_lrf = np.array(f.get("q_sw")) # 100 -> 1000
    t_lw_lrf = np.array(f.get("t_lw"))
    t_sw_lrf = np.array(f.get("t_sw"))

## Load LRF for w
with h5py.File("/home/b11209013/2025_Research/Obs/Files/ERA5/LRF_w.h5", "r") as f:
    w_lw_lrf = np.array(f.get("LW_LRF"))
    w_sw_lrf = np.array(f.get("SW_LRF"))
    eof = np.array(f.get("EOF"))


# load regressed moisture profile
q_reg = np.loadtxt("/work/b11209013/2025_Research/MSI/Rad_Stuff/mean_corr_moisture.txt")
# 100 -> 1000

# Load vertical modes
with h5py.File("/home/b11209013/2025_Research/Kuang2008/data/vertical_mode.h5", "r") as f:
    G1 = np.array(f.get("G1")).squeeze() #1000 -> 100
    G2 = np.array(f.get("G2")).squeeze()
    z  = np.array(f.get("z"))

# Load background density profile
with h5py.File("/home/b11209013/2025_Research/Kuang2008/data/background.h5","r") as f:
    temp_profile = np.array(f.get("T0")).squeeze(); #
    rho_profile = np.array(f.get("ρ0")).squeeze(); # 1000 -> 100

## Interpolation

In [22]:
levels = np.linspace(100, 1000, 37)
z_std = np.array(pressure_to_height_std((levels).astype(int) * units.hPa)) *1000.0

# interpolate moisture profile
mean_moist_itp = np.interp(
            np.linspace(100, 1000, 37), np.array([250, 500, 700, 850, 925, 1000]),
            q_reg) # 100 -> 1000

# Interpolate vertical mode
G1_itp = np.interp(z_std[::-1], z, G1)# *0.0065;  1000 -> 100
G2_itp = np.interp(z_std[::-1], z, G2)# *0.0065;  1000 -> 100

# Interpolate density
rho_profile_itp = np.interp(z_std[::-1], z, rho_profile)[:, None] # 1000 -> 100

## Apply LRF to generate radiative heating

### Convert basis with EOFs

In [23]:
# convert G1 and G2 to omega field
G1_omega = -G1 * rho_profile * 9.81 # 1000 -> 100
G2_omega = -G2 * rho_profile * 9.81

G1_omega_itp = np.interp(z_std[::-1], z, G1_omega) # 1000 -> 100
G2_omega_itp = np.interp(z_std[::-1], z, G2_omega) 

# decompose vertical mode into PCs
print(G1_omega_itp.shape)

G1_pcs = G1_omega_itp[None, :] @ eof.T @ np.linalg.inv(eof @ eof.T)
G2_pcs = G2_omega_itp[None, :] @ eof.T @ np.linalg.inv(eof @ eof.T)


(37,)


### Compute radiation with LRF

In [24]:
# compute radiation for moisture
lw_moisture = (q_lw_lrf @ mean_moist_itp[:,None])[::-1] # 1000 -> 100
sw_moisture = (q_sw_lrf @ mean_moist_itp[:,None])[::-1]

# compute radiation for temperature
lw_temp1 = t_lw_lrf[::-1, ::-1] @ (G1_itp[:,None]*0.0065)
sw_temp1 = t_sw_lrf[::-1, ::-1] @ (G1_itp[:,None]*0.0065)
lw_temp2 = t_lw_lrf[::-1, ::-1] @ (G2_itp[:,None]*0.0065) # 1000 -> 100
sw_temp2 = t_sw_lrf[::-1, ::-1] @ (G2_itp[:,None]*0.0065)

# compute cloud radiation with vertical motion

lw_cld1 = ( (w_lw_lrf @ G1_pcs.T).T @ eof).T
sw_cld1 = ( (w_sw_lrf @ G1_pcs.T).T @ eof).T # 1000 -> 100
lw_cld2 = ( (w_lw_lrf @ G2_pcs.T).T @ eof).T
sw_cld2 = ( (w_sw_lrf @ G2_pcs.T).T @ eof).T
print("Shapes:", lw_cld1.shape, sw_cld1.shape, lw_cld2.shape, sw_cld2.shape)

Shapes: (37, 1) (37, 1) (37, 1) (37, 1)


## Decompose heating to obtain Coefficients

In [25]:
# stack different modes
modes = np.stack([G1_itp*0.0065, G2_itp*0.0065], axis=-1)
print(modes.shape)

lw_moisture1, lw_moistur2 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * lw_moisture))
sw_moisture1, sw_moistur2 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * sw_moisture))

lw_temp11, lw_temp12 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * lw_temp1))
sw_temp11, sw_temp12 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * sw_temp1))
lw_temp21, lw_temp22 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * lw_temp2))
sw_temp21, sw_temp22 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * sw_temp2))

lw_cld11, lw_cld12 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * lw_cld1))
sw_cld11, sw_cld12 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * sw_cld1))
lw_cld21, lw_cld22 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * lw_cld2))
sw_cld21, sw_cld22 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * sw_cld2))

(37, 2)


In [26]:
print("moisture-LW: ", lw_moisture1, lw_moistur2)
print("moisture-SW: ", sw_moisture1, sw_moistur2)
print("temp1-LW: ", lw_temp11, lw_temp12)
print("temp1-SW: ", sw_temp11, sw_temp12)
print("temp2-LW: ", lw_temp21, lw_temp22)
print("temp2-SW: ", sw_temp21, sw_temp22)
print("cloud1-LW: ", lw_cld11, lw_cld12)
print("cloud1-SW: ", sw_cld11, sw_cld12)
print("cloud2-LW: ", lw_cld21, lw_cld22)
print("cloud2-SW: ", sw_cld21, sw_cld22)


moisture-LW:  [4.17550002] [11.07540467]
moisture-SW:  [2.51911098] [-5.32415008]
temp1-LW:  [-0.02133584] [-0.00319572]
temp1-SW:  [6.04474362e-05] [-0.00017112]
temp2-LW:  [-0.0046149] [-0.03789605]
temp2-SW:  [8.78153397e-05] [5.97987274e-05]
cloud1-LW:  [3416.61448276] [7331.88099568]
cloud1-SW:  [2269.21168782] [-3748.63078839]
cloud2-LW:  [-1395.50565593] [-2577.27539799]
cloud2-SW:  [-162.58112066] [1627.1190477]


## Verification

In [27]:
def plot(
        exact_lw,
        exact_sw,
        approx_lw,
        approx_sw,
        levels, mode
):
    max_limit = np.max(
        [
            np.max(np.abs(exact_lw[1:])),
            np.max(np.abs(approx_lw[1:])),
            np.max(np.abs(exact_sw[1:])),
            np.max(np.abs(approx_sw[1:])),
        ]
    )

    fig = plt.figure(figsize=(6,8))
    plt.plot(exact_lw, levels, color="k", linestyle="-", label="Exact LW")
    plt.plot(approx_lw, levels, color="k", linestyle="--", label="Approx LW")
    plt.plot(exact_sw, levels, color="#1f77b4", linestyle="-", label="Exact SW")
    plt.plot(approx_sw, levels, color="#1f77b4", linestyle="--", label="Approx SW")
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.minorticks_on()
    plt.xlabel("Radiative Heating Rate (K/day)")
    plt.ylabel("Pressure (hPa)")
    plt.xlim(-max_limit*1.1, max_limit*1.1)
    plt.ylim(1000, 100)
    plt.legend(frameon=False)
    plt.savefig(f"/home/b11209013/2025_Research/MSI/Fig/Rad/{mode}.png", dpi=300)
    plt.close()

plot(
    lw_moisture * rho_profile_itp, sw_moisture * rho_profile_itp,
    (lw_moisture1*G1_itp*0.0065 + lw_moistur2*G2_itp*0.0065)[:,None],
    (sw_moisture1*G1_itp*0.0065 + sw_moistur2*G2_itp*0.0065)[:,None],
    np.linspace(1000, 100, 37), "moisture"
    )

plot(
    lw_temp1, sw_temp1,
    (lw_temp11*G1_itp*0.0065 + lw_temp12*G2_itp*0.0065)[:,None] / rho_profile_itp,
    (sw_temp11*G1_itp*0.0065 + sw_temp12*G2_itp*0.0065)[:,None] / rho_profile_itp,
    np.linspace(1000, 100, 37), "T1"
    )

plot(
    lw_temp2, sw_temp2,
    (lw_temp21*G1_itp*0.0065 + lw_temp22*G2_itp*0.0065)[:,None] / rho_profile_itp,
    (sw_temp21*G1_itp*0.0065 + sw_temp22*G2_itp*0.0065)[:,None] / rho_profile_itp,
    np.linspace(1000, 100, 37), "T2"
    )

plot(
    lw_cld1, sw_cld1,
    (lw_cld11*G1_itp*0.0065 + lw_cld12*G2_itp*0.0065)[:,None] / rho_profile_itp,
    (sw_cld11*G1_itp*0.0065 + sw_cld12*G2_itp*0.0065)[:,None] / rho_profile_itp,
    np.linspace(1000, 100, 37), "cloud1"
    )

plot(
    lw_cld2, sw_cld2,
    (lw_cld21*G1_itp*0.0065 + lw_cld22*G2_itp*0.0065)[:,None] / rho_profile_itp,
    (sw_cld21*G1_itp*0.0065 + sw_cld22*G2_itp*0.0065)[:,None] / rho_profile_itp,
    np.linspace(1000, 100, 37), "cloud2"
    )